In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
import plotly.express as px
from dash.dependencies import Input, Output, State
from dash import dash_table
import numpy as np
import plotly.graph_objects as go


In [2]:
df = pd.read_csv("train.csv")
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [3]:
#data cleaning missing values
df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [4]:
colors = {
    'background': '#575757',
    'text': 'white'
}
font_size=15

In [5]:
# Data Trasfomation

df['Engine'] = df['Engine'].map(lambda x:str(x)[:-2])
df['Power'] = df['Power'].map(lambda x:str(x)[:-3])
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998,58.16,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582,126.2,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199,88.7,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248,88.76,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968,140.8,5.0,NaN,17.74


In [6]:
#Dealing with missing value
df=df.drop(columns=['New_Price'],axis=1)
df[['Seats']]=df[['Seats']].fillna(5)
df.Engine=pd.to_numeric(df.Engine, errors='coerce')
df.Power=pd.to_numeric(df.Power, errors='coerce')
engin_cc_mean=df.Engine.mean()
power_mean=df.Power.mean()
df[['Engine']]=df[['Engine']].fillna(engin_cc_mean)
df[['Power']]=df[['Power']].fillna(engin_cc_mean)
df.isnull().sum() 

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              2
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             6019 non-null   float64
 9   Power              6019 non-null   float64
 10  Seats              6019 non-null   float64
 11  Price              6019 non-null   float64
dtypes: float64(4), int64(2), object(6)
memory usage: 564.4+ KB


In [8]:
def owner_mean_prices(df):
    owner_mean_price=df[['Owner_Type',"Price"]].groupby('Owner_Type')[
    'Price'].agg([min,max,np.mean]).sort_values(by='mean',ascending=False).reset_index(
    ).rename(columns={'mean':'Mean_Price','min':"Min_Price",'max':'Max_Price'})

    return owner_mean_price
owner_mean_price=owner_mean_prices(df)

In [9]:
search_box=df[['Name','Year','Owner_Type','Price']].groupby(['Name','Year','Owner_Type'])['Price'].agg([max,min,np.mean]).reset_index()

search_box

,Name,Year,Owner_Type,max,min,mean
0,Ambassador Classic Nova Diesel,2003,Third,1.35,1.35,1.35
1,Audi A3 35 TDI Attraction,2014,First,16.50,16.50,16.50
2,Audi A3 35 TDI Premium,2016,First,19.25,19.25,19.25
3,Audi A3 35 TDI Premium Plus,2016,First,18.90,18.90,18.90
4,Audi A3 35 TDI Technology,2017,Second,22.50,22.50,22.50
...,...,...,...,...,...,...
4213,Volvo XC60 D5,2011,First,13.80,12.00,12.90
4214,Volvo XC60 D5,2015,First,32.50,32.50,32.50
4215,Volvo XC60 D5 Inscription,2014,First,17.18,17.18,17.18
4216,Volvo XC90 2007-2015 D5 AT AWD,2014,First,23.58,23.58,23.58


In [10]:
def type_sep(typ,data):
    if typ=='ALL' :
        type_data=data
    elif typ=='OPT1':
        type_data=data[data['Transmission']=='Manual']
    else:
        type_data=data[data['Transmission']=='Automatic']
    return type_data



In [11]:
def Fuel_Type_sep (Fuel_Type,data):
    if Fuel_Type=='CNG':
        fuel_filterd_data=data[data['Fuel_Type']=='CNG']
    elif Fuel_Type=='Diesel':
        fuel_filterd_data=data[data['Fuel_Type']=='Diesel']
    elif Fuel_Type=='ALL':
        fuel_filterd_data=data
    elif Fuel_Type=='Petrol':
        fuel_filterd_data=data[data['Fuel_Type']=='Petrol']
    elif Fuel_Type=='LPG':
        fuel_filterd_data=data[data['Fuel_Type']=='LPG']
    else:
        fuel_filterd_data=data[data['Fuel_Type']=='Electric']
    return fuel_filterd_data

In [12]:
def graph_creater(data):
    fig=[  px.box(data,x='Owner_Type',y='Price',title='Owner Type vs. Price',color='Transmission'),
           px.box(data,x='Seats',y='Price',title='Seats vs. Price',color='Transmission'),
           px.histogram(data,x='Location',color='Transmission',title='Vehicle Count in Each City')]
    for i in fig:
        i.update_layout(plot_bgcolor=colors['background'],
                      paper_bgcolor=colors['background'],
                      font_color=colors['text'],width = 550,
        height = 400,
                       )
        
    return fig

In [13]:
def Trasmition_pie_fig(data):
    Transmission_type_data=data.groupby('Transmission').count().reset_index()[['Transmission','Price']]
    Transmission_type_data=Transmission_type_data.rename({'Price':"Count"},axis=1)
    fig=px.pie(Transmission_type_data,values="Count",names='Transmission',title="Trasmission Types",
               color_discrete_sequence=px.colors.sequential.RdBu,hole=.3)
    fig.update_layout(plot_bgcolor=colors['background'],
                      paper_bgcolor=colors['background'],width = 400,
        height = 400,
                      font_color=colors['text'],
                    )
    
    return fig

In [14]:
def fig_creater(data,col_name1,col_name2):
    
    fig=[px.histogram(data,x=col_name1,title=("Distribution of "+col_name1)),
         px.histogram(data,x='Owner_Type',title="Count of the Owners" ),
        px.scatter(df,x=col_name2,y='Price',title=('Price vs '+col_name2))]
    for i in fig:
        i.update_layout(plot_bgcolor=colors['background'],
                      paper_bgcolor=colors['background'],
                      font_color=colors['text'], width = 450,
        height = 400,template='seaborn'

                      )
    return fig

In [15]:
def year_fillter(data,year):
    if year=="ALL":
        year_filterd_data=df
    else:
        year_filterd_data=df[df['Year']==int(year)]
    return year_filterd_data

In [16]:
app=dash.Dash(__name__)

In [17]:
year_list=sorted(list(df.Year.unique()))
year=[{'label': i, 'value': i} for i in year_list]
year.append({'label':'ALL','value': 'ALL'})
row=df.Fuel_Type.unique()
fuel_type=[{'label': i, 'value': i} for i in row]
fuel_type.append({'label':'ALL','value': 'ALL'})
owner_type=list(df.Owner_Type.unique())
heat_map_data1=df.corr()
columns_names=[{'label':'Kilometers_Driven','value':'Kilometers_Driven'},
                {'label':'Engine','value':'Engine'},
                {'label':'Power','value':'Power'},
                {'label':'Price','value':'Price'},
               {'label':'Year','value':'Year'}]
heat_map_data=heat_map_data1[['Kilometers_Driven','Engine','Power','Price']].drop(index=['Year','Seats'])

In [18]:
app.layout=html.Div(children=[html.H1('USED CAR MARKET DASHBOARD',style={'textAlign':'center',
                                        'color':colors['text'],'font-size':30}),
                               
                                html.Div([
                                    
                                     
                                      
                                   html.Div([
                                       
                                        html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em','color':colors['text'],'font-size': '14px'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-year', 
                                                    
                                                     options=year,
                                                     value="ALL",

                                                     placeholder="Select a year",
                                                     style={'width':'300px', 'font-size': '14px', 'text-align-last' : 'center'}),
                                            
                                           html.Div(
                                            [
                                            html.H2('Fuel Type:', style={'margin-right': '2em','margin-left': '3em','color':colors['text'],'font-size': '14px'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input_Fuel_Type', 
                                                    
                                                     options=fuel_type,
                                                     value="ALL",

                                                     placeholder="Select a year",
                                                     style={'width':'300px', 'padding':'2px', 'font-size': '14px', 'text-align-last' : 'center'}),
                                       html.Div(
                                            [
                                           
                                                html.H2('Trasmission Type:', style={'margin-right': '2em','margin-left': '3em','color':colors['text'],'font-size': '14px'}),
                                            ]
                                        ),dcc.Dropdown(
                                    id='input_vehical_type',
                                    options=[
                                        {'label': 'Manual', 'value': 'OPT1'},
                                        {'label': 'Automatic', 'value': 'OPT2'},
                                        {'label': 'ALL', 'value': 'ALL'}
                                    ],value="ALL",
                                    
                                   placeholder="Select The Type",
                                    style={'width':'300px', 'padding':'2px', 'font-size': '14px', 'text-align-last' : 'center'})
                                   ], style={'display': 'flex'}),  
                                   ]),
                          
                        
                               html.Div([
                                   
                                  html.Div([], id='plot1',), 
                                             
                                   
                                         
                                   html.Div([], id='plot2',),
                                   html.Div([], id='plot3',),
                                   
#                                    
                               ],style={'display': 'flex',}),
                              
                               
                               
                                                        
                              
                              
                              
                               html.Div([
                                
                                   
#                                html.Div([
#                               dash_table.DataTable(
#                                     id='table',
#                                     columns=[{"name": i, "id": i} for i in owner_mean_price.columns],
#                                     data=owner_mean_price.to_dict('records'),
#                                style_data={'color': 'black','backgroundColor': 'white'},
                                  
#                                style_data_conditional=[{'if': {'row_index': 'odd'},'backgroundColor': 'rgb(220, 220, 220)',}],
#                                 style_header={'backgroundColor': 'rgb(210, 210, 210)','color': 'black','fontWeight': 'bold'
#     })
#                                         ],style={'width':'800px'},)
                              ]),
                              html.Div([
                                  html.Div([dcc.RadioItems(
                                id='column_name_change',
                                options=[{'label':'Kilometers_Driven','value':'Kilometers_Driven'},
                {'label':'Engine','value':'Engine'},
                {'label':'Power','value':'Power'},
                {'label':'Price','value':'Price'},
               {'label':'Year','value':'Year'}],
                                    value='Year',style={'width':'400px', 'padding':'3px','color':colors['text']},
                                   labelStyle={'display': 'inline-block'}
                                    ),
                              
                                  
                                       html.Div([], id='fig1'),
                                  ])
                                  ,
                                      html.Div([], id='fig2'),
                                      html.Div([],id='plot4')
                                  

   

    
                ],style={'display': 'flex',}),
                              
                             html.Div([
                                  html.Div([
                                dcc.Checklist(
                                id='heat_slt_col',
                                options=[{'label': x, 'value': x} 
                                     for x in heat_map_data.columns],
                                    value=heat_map_data.columns.tolist(),
                                    style={'width':'500px', 'padding':'3px','color':colors['text']}
                                    ),
                                dcc.Graph(id="Heat_map")]),
                                html.Div([
                                            html.H2('VehicleType:', style={'margin-right': '2em',
                                                                           'color':colors['text'],'font-size': '14px'}),
                                         
                                    dcc.Dropdown(id='choice2',
                                         options=[{'label':x, 'value':x}
                                                  for x in sorted(df.Name.unique())],
                                         value='Maruti Wagon R LXI CNG',
                                            style={'width':'300px', 'padding':'3px'}
                                         ),
                        #Initialing the graph from dash core components
                                          dcc.Graph(id="graph2")]),
                                 html.Div([dcc.RadioItems(
                                id='column_name_change2',
                                options=[{'label':'Kilometers_Driven','value':'Kilometers_Driven'},
                {'label':'Engine','value':'Engine'},
                {'label':'Power','value':'Power'},
    
                               ],
                                    value='Kilometers_Driven',style={'width':'500px', 'padding':'4px','color':colors['text']},
                                   labelStyle={'display': 'inline-block'}
                                    ),
                              
                                  
                                       html.Div([], id='fig3'),
                                  ],style={'margin-left': '6em'})
                        ],style={'display': 'flex',}),   
                                ],style={'background-color':colors['background']})
                            
                            
                                
                           
                               
                              
                 

In [19]:
@app.callback( [Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
                Output(component_id='plot3', component_property='children'),
                Output(component_id='plot4', component_property='children')
               
               ],
               [Input(component_id='input-year', component_property='value'),
               Input(component_id='input_vehical_type', component_property='value'),
               Input(component_id='input_Fuel_Type', component_property='value')],
              [State("plot1", 'children'),State("plot2", 'children'),State("plot3", 'children'),
               State("plot4", 'children')
               ])
def graph_render(year,vehical_type,Fuel_Type,c1,c2,c3,c4):
    
    year_filterd_data=year_fillter(df,year)
   
    fuel_and_year_filterd_data=Fuel_Type_sep(Fuel_Type,year_filterd_data)
    
    tras_fuel_year_sep_data=type_sep(vehical_type,fuel_and_year_filterd_data)
    
    
    pie_fig=Trasmition_pie_fig(fuel_and_year_filterd_data)
    
    box_fig_owner,box_fig_seats,stck_fig=graph_creater(tras_fuel_year_sep_data)
    
    return [dcc.Graph(figure=pie_fig),
            dcc.Graph(figure=box_fig_owner),
           dcc.Graph(figure=box_fig_seats),
           dcc.Graph(figure=stck_fig)]


In [20]:
@app.callback(
    Output("Heat_map", "figure"), 
    [Input("heat_slt_col", "value")])
def filter_heatmap(cols):
    fig = px.imshow(heat_map_data[cols] , title = "Heat Map")
    fig.update_layout(plot_bgcolor=colors['background'],
                      paper_bgcolor=colors['background'],width = 400,
                      font_color=colors['text'],
                     )
    return fig

In [21]:
@app.callback( [Output(component_id='fig1', component_property='children'),
                Output(component_id='fig2', component_property='children'),
                Output(component_id='fig3', component_property='children')
               ],
               [Input(component_id='input_Fuel_Type', component_property='value'),
               Input(component_id='column_name_change', component_property='value'),
               Input(component_id='column_name_change2', component_property='value')],
              [State("fig1", 'children'),State("fig2", 'children'),State("fig3", 'children')])

def get_graph(Fuel_Type,col_name1,col_name2,c1,c2,c3):
    
    filt_data=Fuel_Type_sep(Fuel_Type,df)
  
    fig1,fig2,fig3=fig_creater(filt_data,col_name1,col_name2)
    
    
    
    
    
    
    return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2),
            dcc.Graph(figure=fig3)
            ]

In [22]:
@app.callback(
    Output("graph2","figure"),
    [Input("choice2","value")]
   
)
def update_figure(VehicleType):
    dff = df[df.Name==VehicleType]
    fig=px.pie(dff, names='Location',title='Sold or Available Locations of Cars')
    fig.update_layout(plot_bgcolor=colors['background'],
                      paper_bgcolor=colors['background'],width = 400,
        height = 400,
                      font_color=colors['text'],
                    )
    return fig

In [23]:
if __name__=='__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Feb/2022 12:04:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:44] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2022 12:04:46] "POST /_dash-update-component HTTP/1.1" 